# Passive membrane parameters

Experiments carried out at 29C to 30C by Michael Lynn in spring 2022.

In [1]:
import os
os.chdir(os.path.join('..', '..'))
print(os.getcwd())

/home/efharkin/projects/raphegif/raphegif


In [2]:
import pandas as pd
import numpy as np

In [3]:
def get_membrane_params_from_all_csv_files(dir_to_search):
    index_files = [
        os.path.join(dirpath, filename)
        for (dirpath, dirs, files) in os.walk(dir_to_search)
        for filename in (dirs + files)
        if filename.lower().endswith('.csv') and 'heated' in dirpath.lower()
    ]
    tables = [
        pd.read_csv(path, comment='#').loc[
            :,
            [
                'cell_id',
                'membrane_resistance_megaohm',
                'membrane_capacitance_picofarad',
            ],
        ]
        for path in index_files
    ]
    master_table = pd.concat(tables, ignore_index=True)
    return master_table

In [4]:
ser_master_table = get_membrane_params_from_all_csv_files(
    os.path.join(os.getenv('DATA_PATH'), 'raw', '5HT')
)

# Remove duplicate cells.
# First, find duplicate cells by ID, then check that each duplicate has exactly
# the same values for membrane parameters, and finally average across
# duplicates to arrive at a table with one set of parameters per cell ID.
for col in ser_master_table:
    if col == 'cell_id':
        continue
    __std = ser_master_table.groupby('cell_id').std()[col]
    assert all(np.isclose(__std, 0.0) | np.isnan(__std)), '{}'.format(col)
ser_master_table = ser_master_table.groupby(
    'cell_id'
).mean()

ser_master_table['leak_conductance_nanosiemen'] = (
    1e3 / ser_master_table['membrane_resistance_megaohm']
)

ser_master_table['membrane_time_constant_millisecond'] = (
    ser_master_table['membrane_resistance_megaohm']
    * ser_master_table['membrane_capacitance_picofarad']
    * 1e-3  # Convert time constant to ms from s.
)
ser_master_table

,membrane_resistance_megaohm,membrane_capacitance_picofarad,leak_conductance_nanosiemen,membrane_time_constant_millisecond
cell_id,,,,
DRN1008,250.0,20.0,4.000000,5.00
DRN1010,460.0,60.0,2.173913,27.60
DRN1011,450.0,30.0,2.222222,13.50
DRN1012,350.0,27.0,2.857143,9.45
DRN1014,220.0,50.0,4.545455,11.00
DRN1015,600.0,115.0,1.666667,69.00
DRN1016,500.0,80.0,2.000000,40.00
DRN1018,400.0,65.0,2.500000,26.00
DRN1059,900.0,44.0,1.111111,39.60


In [5]:
ser_master_table.mean()

membrane_resistance_megaohm           552.962963
membrane_capacitance_picofarad         57.629630
leak_conductance_nanosiemen             2.141723
membrane_time_constant_millisecond     32.468519
dtype: float64

In [6]:
ser_master_table.sem()

membrane_resistance_megaohm           46.097923
membrane_capacitance_picofarad         3.916561
leak_conductance_nanosiemen            0.172929
membrane_time_constant_millisecond     3.258824
dtype: float64

In [7]:
ser_master_table.std()

membrane_resistance_megaohm           239.531832
membrane_capacitance_picofarad         20.351050
leak_conductance_nanosiemen             0.898564
membrane_time_constant_millisecond     16.933349
dtype: float64

In [8]:
ser_master_table.shape

(27, 4)